In [ ]:
import os
#os.environ["OPENAI_API_KEY"]=""
os.environ["OPENAI_API_KEY"]=" insert your api key" 

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools.tool_spec.base import BaseToolSpec
from typing import List

# Step 1: Define a simple custom tool
class MathTool(BaseToolSpec):
    spec_functions = ["add", "sub", "mul", "div"]

    def add(self, a: int, b: int) -> int:
        """Add two numbers"""
        return a + b

    def sub(self, a: int, b: int) -> int:
        """Subtract two numbers"""
        return a - b

    def mul(self, a: int, b: int) -> int:
        """Multiply two numbers"""
        return a * b

    def div(self, a: int, b: int) -> float:
        """Divide two numbers"""
        return a / b

# Step 2: Load document and create index
f_path = "D:/RAMANASOFT/alice-in-wonderland.pdf"
reader = SimpleDirectoryReader(input_files=[f_path])
pdf_documents = reader.load_data()
index = VectorStoreIndex.from_documents(pdf_documents)
query_engine = index.as_query_engine()

# Step 3: Document tool wrapper
class DocumentTool(BaseToolSpec):
    spec_functions = ["ask_doc"]

    def ask_doc(self, question: str) -> str:
        """Answer questions based on the document"""
        return str(query_engine.query(question))

# Step 4: Set up LLM
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# Step 5: Generate tools from specs
tools = DocumentTool().to_tool_list() + MathTool().to_tool_list()

# Step 6: Create agent
agent = OpenAIAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
)

# Step 7: Run chat
if __name__ == "__main__":
    while True:
        query = input("Ask something: ")
        if query.lower() in ["exit", "quit"]:
            break
        response = agent.chat(query)
        print(f"Response: {response}")


Ask something:  who is alice


Added user message to memory: who is alice
=== Calling Function ===
Calling function: ask_doc with args: {"question":"Who is Alice?"}
Got output: Alice is a character in the book "Alice in Wonderland."

Response: Alice is a character in the book "Alice in Wonderland."
